# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [22]:
import pandas as pd
import numpy as np
from scipy import stats
import math

In [4]:
data = pd.io.stata.read_stata('/Users/jacobschroeder/anaconda3/projects/racial_disc/data/us_job_market_discrimination.dta')

In [5]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


------------

### 1. What test is appropriate for this problem? Does CLT apply?

A two-sample t-test is appropriate for this problem. We will use CLT to determine measures of center for each sample (black and white, accordingly) and use a t-test to see how the samples overlap with regards to callbacks.

----------

### 2. What are the null and alternate hypotheses?

##### The Null Hypothesis:

The null hypothesis is that blacks and whites get an equal percentage of callbacks. 

##### The Alternate Hypothesis:

The alternate hypothesis is that race plays a role in callbacks, and that blacks and white do not get equal percentage of callbacks

---------------

### 3. Compute margin of error, confidence interval, and p-value.

In [34]:
# import bootstrap replicate functions
def bootstrap_replicate_1d(data, func):
    """Generate bootstrap data of 1D data."""
    bs_sample = np.random.choice(data, len(data))
    return func(bs_sample)


def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates


callbacks_b = data[['race', 'call']][data['race'] == 'b']
# print(callbacks_b)
entries_b, _ = callbacks_b.shape
print('Black entries: {}'.format(entries_b))

callbacks_w = data[['race', 'call']][data['race'] == 'w']
entries_w, _ = callbacks_w.shape
print('White entries: {}'.format(entries_w))



# compute sample size, draw a random sample and compute its mean
sample_size = entries_b
sample = np.random.choice(a=callbacks_b['call'], size = sample_size)
sample_mean = sample.mean()
print("""
        ================================================
        Black sample callback mean: {}
        ================================================
        """.format(sample_mean))


# calculating z_critical: number of standard deviations to get 95% confidence interval
z_critical = stats.norm.ppf(q = 0.975)

print("""
        ================================================
        z-critical value for 95% confidence interval: {}
        ================================================
""".format(z_critical))                    

# get the standard deviation for the popluation as a whole, and for whites
pop_stdev = data['call'].std()
w_stdev = callbacks_w['call'].std()

# compute comparison statistics for blacks vs. population
margin_of_error = z_critical * (pop_stdev/math.sqrt(sample_size))
confidence_interval = (sample_mean - margin_of_error,
                       sample_mean + margin_of_error)  

# p-value calculation for blacks vs. population
callbacks_b_reps = draw_bs_reps(callbacks_b['call'], np.mean, size=10000)
callbacks_b_p_value = sum(callbacks_b_reps >= np.mean(data['call']))/len(callbacks_b_reps)

print("""
        Blacks vs. population summary statistics
        ----------------------------------------
        Margin of Error: {}
        Confidence Interval: {}
        P-Value: {}
        ========================================
""".format(margin_of_error, confidence_interval, callbacks_b_p_value))

# compute comparison statistics for blacks vs. whites
margin_of_error = z_critical * (w_stdev/math.sqrt(sample_size))
confidence_interval = (sample_mean - margin_of_error,
                       sample_mean + margin_of_error)  

# p-value calculation for blacks vs. whites 
# Using same set of bootstrap replicates for comparison
callbacks_b_p_value = sum(callbacks_b_reps >= np.mean(callbacks_w['call']))/len(callbacks_b_reps)

print("""
        Blacks vs. Whites summary statistics
        ----------------------------------------
        Margin of Error: {}
        Confidence Interval: {}
        P-Value: {}
        ========================================
""".format(margin_of_error, confidence_interval, callbacks_b_p_value))

Black entries: 2435
White entries: 2435

        Black sample callback mean: 0.06611909717321396
        

        z-critical value for 95% confidence interval: 1.959963984540054


        Blacks vs. population summary statistics
        ----------------------------------------
        Margin of Error: 0.010806730722971825
        Confidence Interval: (0.055312366450242137, 0.076925827896185781)
        P-Value: 0.0017


        Blacks vs. Whites summary statistics
        ----------------------------------------
        Margin of Error: 0.011730836680568891
        Confidence Interval: (0.054388260492645071, 0.077849933853782846)
        P-Value: 0.0



-----------------

### 4. Write a story describing the statistical significance in the context or the original problem.

##### Findings Explained

Note the p-value measures instances where the average callbacks for blacks met or exceeded callbacks for the comparison sample (population and whites, respectively).

When compared to the population (the dataset containing both races), we reject the null hypothesis that blacks are treated the same given the remarkably low p-value.

This rejection is further validated when the comparison is made directly to whites. The p-value is so small that it fails to register. 

These assertions are both made with a very small (1%) margin of error and relative high confidence (95%).

##### What This Means

It is clear that black-sounding names receive significantly fewer callbacks for job openings than white-sounding names. The analysis provides statistical backing for the assertion that racism is still alive and well when it comes to employment. Whites are more likely to get callbacks, and therefore are granted an advantage when it comes to career opportunites.


--------------------------

### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

While this analysis strongly suggests race is a factor in callback success, whether or not it is the most important factor is left to further investigation. The reason for this amibguity is because there are many other factors (education, experience, military) that may also play a role.

The analysis needs to be ammended to include a Pearson correlation coefficient between the various factors and callback success. These factors would be broken down into "background" and "opportunity" since different backgrounds may lend themselves to more success for particular type of opportunity (for example, perhaps education plays a larger role in banking/real estate, but military background lends itself well to service and manufacturing jobs).

This analysis could be broken down by race, and a two-sample t-test could again be utilized. This way, we could see not only what the most important factors are, but if these factors have greater or less impact depending on race (for example, does education not contribute to success as much for blacks as it does for whites when applying for the same job type?).